In [390]:
import os
import pandas as pd

from importlib import reload
import countries
reload(countries)
if os.getcwd().split('/')[-1] == 'mobility': os.chdir('..')
from countries import FROM_COUNTRY

df = pd.read_csv('dist/static/mobility/world.csv.gz')
df

,country_geoid,region,category,page,change,date,value,country
0,AE,total,retail/recreation,0,-55,2020-02-16,0.857313,United Arab Emirates
1,AE,total,retail/recreation,0,-55,2020-02-17,-1.365351,United Arab Emirates
2,AE,total,retail/recreation,0,-55,2020-02-18,-1.621751,United Arab Emirates
3,AE,total,retail/recreation,0,-55,2020-02-19,-1.549515,United Arab Emirates
4,AE,total,retail/recreation,0,-55,2020-02-20,-1.748761,United Arab Emirates
...,...,...,...,...,...,...,...,...
170083,ZW,total,residential,1,13,2020-03-25,15.087301,Zimbabwe
170084,ZW,total,residential,1,13,2020-03-26,18.100492,Zimbabwe
170085,ZW,total,residential,1,13,2020-03-27,23.208374,Zimbabwe
170086,ZW,total,residential,1,13,2020-03-28,13.050313,Zimbabwe


In [391]:
import altair as alt

country = "France"
category = "workplace"
subset = df.query('country == @country and category == @category')
chart_data = subset.groupby(['date'])['value'].mean().reset_index()
(alt.Chart(chart_data)
 .mark_line(point=True)
 .encode(x='date', y='value')
 +
 alt.Chart(subset.query('region == "total"'))
 .mark_line(point=True)
 .encode(x='date', y='value')
)

In [392]:
(alt.Chart(subset.reset_index())
 .mark_line(point=True)
 .encode(x='date', y='value', color='region', tooltip=['region', 'date', 'value'])
 .properties(width=700)
)

In [393]:
fr = df.query('country == "France" and region == "total"')
alt.Chart(fr).mark_line(point=True).encode(x='date:T', y='value', color='category')

In [394]:
df = df.assign(country_code=df['country'].map(FROM_COUNTRY))
df

,country_geoid,region,category,page,change,date,value,country,country_code
0,AE,total,retail/recreation,0,-55,2020-02-16,0.857313,United Arab Emirates,784
1,AE,total,retail/recreation,0,-55,2020-02-17,-1.365351,United Arab Emirates,784
2,AE,total,retail/recreation,0,-55,2020-02-18,-1.621751,United Arab Emirates,784
3,AE,total,retail/recreation,0,-55,2020-02-19,-1.549515,United Arab Emirates,784
4,AE,total,retail/recreation,0,-55,2020-02-20,-1.748761,United Arab Emirates,784
...,...,...,...,...,...,...,...,...,...
170083,ZW,total,residential,1,13,2020-03-25,15.087301,Zimbabwe,716
170084,ZW,total,residential,1,13,2020-03-26,18.100492,Zimbabwe,716
170085,ZW,total,residential,1,13,2020-03-27,23.208374,Zimbabwe,716
170086,ZW,total,residential,1,13,2020-03-28,13.050313,Zimbabwe,716


In [395]:
from vega_datasets import data

world_topo = alt.topo_feature(data.world_110m.url, 'countries')

In [396]:
alt.data_transformers.enable('default')

categories = list(df.category.unique())
map_data = (df
              .query('region == "total"')
              .set_index(['country', 'country_code', 'region', 'category'])
              [['date', 'value']]
            .assign(value=lambda f: f['value'])
            ['value']
            .div(100)
              .groupby(['country', 'country_code', 'region', 'category'])
              .last()
            .unstack()
              .reset_index())

input_dropdown = alt.binding_select(options=categories)
selection_category = alt.selection_single(fields=['category'], 
                                          bind=input_dropdown, 
                                          name='Mobility',
                                          init={'category': 'workplace'})

selection_country = alt.selection_multi(
        fields=['country'],
        name='Country of',
        empty='all',
#     init={'country': 'France'}
    )

background = (alt.Chart(world_topo)
              .mark_geoshape(fill='lightgray', stroke='white', strokeWidth=0.5)
              .transform_filter('datum.id != 10')
#               .transform_filter('datum.id != 304')
             )

foreground = (alt.Chart(world_topo)
        .mark_geoshape(stroke='white', strokeWidth=0.5)
         .encode(
             color=alt.condition(
                 selection_country,
                 alt.Color('value:Q', scale=alt.Scale(scheme='blueorange', domainMid=0), legend=alt.Legend(format=".0%")),
                 alt.value('lightgray')
             ),
             tooltip=[alt.Tooltip('value:Q', format='.0%'), 'country:N']
         )
            .transform_lookup(
                lookup='id',
                from_=alt.LookupData(data=map_data,
                                 key='country_code', 
                                 fields=['country'] + categories)
            )
            .transform_fold(
                fold=categories,
                as_=['category', 'value']
            )
             .add_selection(selection_category)
             .transform_filter(selection_category)
            ).add_selection(selection_country)

map_chart = (background + foreground).properties(width=700, height=500, 
#                                                  title="Variation to baseline on March 29"
                                                )

ts_data = df.query("region == 'total'").assign(value=lambda f: f['value'].div(100))
base_ts = alt.Chart(ts_data)

ts_chart = (base_ts
            .mark_line(point=True)
            .encode(x='date:T', 
                    y=alt.Y('value:Q', axis=alt.Axis(format='%')), 
                    tooltip=[alt.Tooltip('date:T', format='%a, %b %e'), alt.Tooltip('value:Q', format='.1%')])
#             .properties(title='Variation through time')
         .add_selection(selection_category)
            .add_selection(selection_country)
            .transform_filter(selection_category)
            .transform_filter(selection_country)
            .transform_aggregate(
                value='mean(value)',
                groupby=['category', 'date']
            )
        )

chart = (ts_chart | map_chart).properties(title='Mobility change by geography, across different categories of places (Variation to baseline on March 29)')
chart.save('dist/static/mobility/charts/map.html')
alt.data_transformers.enable('data_server')
chart

In [318]:
subset = (df.query('country == "France" and region == "total"')
          .assign(date=lambda f: pd.to_datetime(f['date']))
          .assign(weekday=lambda f: f['date'].dt.dayofweek, 
                  weekofyear=lambda f: f['date'].dt.weekofyear,
                  is_weekend=lambda f: f['weekday'].isin([5, 6]).astype(int).astype(str),
                  time=lambda f: f['weekofyear'].astype(str).str.pad(2, fillchar='0').str.cat(f['is_weekend'])
                 )
)
subset

,country_geoid,region,category,page,change,date,value,country,country_code,weekday,weekofyear,is_weekend,time
51976,FR,total,retail/recreation,0,-88,2020-02-16,5.602760,France,250,6,7,1,071
51977,FR,total,retail/recreation,0,-88,2020-02-17,1.075916,France,250,0,8,0,080
51978,FR,total,retail/recreation,0,-88,2020-02-18,4.737228,France,250,1,8,0,080
51979,FR,total,retail/recreation,0,-88,2020-02-19,-3.767704,France,250,2,8,0,080
51980,FR,total,retail/recreation,0,-88,2020-02-20,-1.635872,France,250,3,8,0,080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52229,FR,total,residential,1,18,2020-03-25,31.331890,France,250,2,13,0,130
52230,FR,total,residential,1,18,2020-03-26,33.617282,France,250,3,13,0,130
52231,FR,total,residential,1,18,2020-03-27,36.963806,France,250,4,13,0,130
52232,FR,total,residential,1,18,2020-03-28,24.261902,France,250,5,13,1,131


In [324]:
(subset
 .groupby(['category', 'weekofyear'])
 .mean()
 .reset_index()
 .pipe(alt.Chart)
 .mark_line()
 .encode(x='weekofyear', y='value', color='category')
)

In [378]:
ecdc = pd.read_csv('mobility/ecdc.csv')
ecdc

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,04/04/2020,4,4,2020,0,0,Afghanistan,AF,AFG,37172386.0
1,03/04/2020,3,4,2020,43,0,Afghanistan,AF,AFG,37172386.0
2,02/04/2020,2,4,2020,26,0,Afghanistan,AF,AFG,37172386.0
3,01/04/2020,1,4,2020,25,0,Afghanistan,AF,AFG,37172386.0
4,31/03/2020,31,3,2020,27,0,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
8699,25/03/2020,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
8700,24/03/2020,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
8701,23/03/2020,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
8702,22/03/2020,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


In [389]:
ecdc.groupby(['geoId', 'popData2018']).first().reset_index()[['geoId', 'popData2018']].head(60)

,geoId,popData2018
0,AD,7.700600e+04
1,AE,9.630959e+06
2,AF,3.717239e+07
3,AG,9.628600e+04
4,AL,2.866376e+06
5,AM,2.951776e+06
6,AO,3.080976e+07
7,AR,4.449450e+07
8,AT,8.847037e+06
9,AU,2.499237e+07
